<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/cpp_functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
functional programming in c++ をなぞり読み

https://www.manning.com/books/functional-programming-in-c-plus-plus#toc

In [ ]:
!sudo apt install librange-v3-dev

In [ ]:
# 最初の例
# 命令型の例
# main のないセルを実行するとエラーになる
# コメントアウトするか %%script false としておく
# Listing 1.1 Calculating the number of lines the imperative way
%%script false
vector<int> count_lines_in_files(const vector<string>& files) {
    vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        ifstream in(file); 
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}

In [ ]:
# 動く形にして見よう
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

vector<int> count_lines_in_files(const vector<string>& files) {
    vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        ifstream in(file); 
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}

int main() {
    ofstream ofs01("./temp01.txt");
    ofs01 << "foo bar baz \nThis is a pen" << endl;
    ofs01.close(); // 2行
    ofstream ofs02("./temp02.txt");
    ofs02 << "日本語もどうか" << endl << endl;
    ofs02 << "これが最後" << endl << "ん" << endl; 
    ofs02.close(); // 4行

    vector<string> files = {"temp01.txt", "temp02.txt"};
    vector<int> lines;
    lines = count_lines_in_files(files);
    for(int i : lines) {
        cout << i << endl;
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# とりあえず動いた
# "\n" を数えているのでファイル末に改行がないと正しい数字にならない
# getline を使いたい

In [ ]:
# Listing 1.2 Using std::count to count newline characters
# count version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  ifstream in(filename);
  return count(istreambuf_iterator<char>(in), istreambuf_iterator<char>(), '\n');
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results;
  for (const auto& file : files) {
    results.push_back(count_lines(file));
  }
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# count version は count を使っている事より、istreambuf_iterator を知らなかった
# cpprefjp による例
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int main()
{
  stringstream ss;
  ss << "1 2 3" << endl
     << "4 5 6";

  // 文字列の入力ストリームから順に文字を読み込むイテレータを用意
  istreambuf_iterator<char> it(ss);
  istreambuf_iterator<char> last;

  // イテレータを進めることにより、入力ストリームからデータを順に読み取る
  for_each(it, last, [](char c) { std::cout << c; });
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

1 2 3
4 5 6

In [ ]:
# ちょっとよくわかりません
# とりあえず getline version を作って、ファイル末の改行なしに対応するか実験しよう #=> 成功!!!!
# getline version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results;
  for (const auto& file : files) {
    results.push_back(count_lines(file));
  }
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# テキストは次に transform を使っている
# transform は他の言語では map に当たるアルゴリズムである
# map -> transform, reduce -> accumulate, filter -> copy_if らしい
# transform version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<std::string>& files) {
    vector<int> results(files.size());
    // transform(files.cbegin(), files.cend(), results.begin(), count_lines);    
    transform(files.begin(), files.end(), results.begin(), count_lines); // cbegin と begin の違いは const かどうかでほとんど同じ
    return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# ranges version
# ranges は chapter 7 で説明するとのこと
# だがエラーなのでとりあえず %%script false にして先へ進むか
# 実験
# !sudo apt install librange-v3-dev
# include <range/v3/view/transform.hpp>
# ::ranges::view::transform
# としたら動いた
%%writefile temp.cpp
# include <bits/stdc++.h>
# include <range/v3/view/transform.hpp>

using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<string>& files) {
    return files | ::ranges::view::transform(count_lines);
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Writing temp.cpp


In [ ]:
!sudo apt install librange-v3-dev

In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# 最終形は次のようになる、とのこと
# transform を 2回に分けている
%%script false
vector<int>
count_lines_in_files(const vector<string>& files) {
    return files | transform(open_file) | transform(count_lines);
}

In [ ]:
# Listing 2.1 Calculating the average score imperatively
# 平均を算出する 命令形
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

double average_score(const vector<int>& scores) {
    int sum = 0;    
    for (int score : scores) {    
        sum += score;    
    }    
    return sum / (double)scores.size();    
}

int main() {
    vector<int> vec = {9, 7, 10, 5, 8, 8, 6};
    double average;
    average = average_score(vec);
    cout << average << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

7.57143


In [ ]:
# Listing 2.2 Calculating the average score functionally
# 関数型
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

double average_score(const vector<int>& scores) {
    return accumulate(scores.cbegin(), scores.cend(),0) / (double)scores.size();    
}

int main() {
    vector<int> vec = {9, 7, 10, 5, 8, 8, 6};
    double average;
    average = average_score(vec);
    cout << average << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

7.57143


In [ ]:
# PARALLEL VERSIONS OF STANDARD ALGORITHMS
#  -std=c++17 にしても reduce , execution が使えないので %%script false
# 並行処理は一旦諦める
%%script false
%%writefile temp.cpp
# include <bits/stdc++.h>

using namespace std;

double average_score(const vector<int>& scores) {
    return reduce(execution::par, scores.cbegin(), scores.cend(),0) / (double) scores.length();
}

int main() {
    vector<int> vec = {9, 7, 10, 5, 8, 8, 6};
    double average;
    average = average_score(vec);
    cout << average << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ -std=c++17 temp.cpp

In [ ]:
# c++17 の any は -std=c++17 して include  すれば使える
# reduce や execution は同じようにしてもエラーになる
%%writefile temp.cpp
# include <bits/stdc++.h>
# include <any>
using namespace std;

int main(){
    auto s = any(123);
    cout << any_cast<int>(s) << endl; // "123"が標準出力
}

Overwriting temp.cpp


In [ ]:
!g++ -std=c++17 temp.cpp; ./a.out

123


In [ ]:
# Listing 2.3 Calculating the product of all scores
# accumulate の使い方 関数が渡せる fold や reduce みたい
# この例では multiplies<int>() というのを渡している
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

double scores_product(const vector<int>& scores) {
  return accumulate(scores.cbegin(), scores.cend(), 1, multiplies<int>());
}

int main() {
  vector<int> vec = { 2,3,4 };
  cout << scores_product(vec) << endl;  //=> 24
}

Overwriting temp.cpp


In [ ]:
!g++  temp.cpp; ./a.out

24


In [66]:
# folding
# Counting newline characters with accumulate
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int f(int previous_count, char c) {
  return (c != '\n') ? previous_count : previous_count + 1;
}

int count_lines(const string& s) {
  return accumulate(s.cbegin(), s.cend(), 0, f);
}

int count_lines1(const string& filename) {
  ifstream in(filename);
  stringstream myStream;
  myStream << in.rdbuf();
  return count_lines(myStream.str());
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results(files.size());
  transform(files.begin(), files.end(), results.begin(), count_lines1);
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [67]:
!g++  temp.cpp; ./a.out

2
3


In [68]:
# 2.2.3 String trimming
%%script false
string trim_left(string s) {
    s.erase(s.begin(), find_if(s.begin(), s.end(), is_not_space));
    return s;
}

In [ ]:
%%script false
string trim_right(string s) {
    s.erase(find_if(s.rbegin(), s.rend(), is_not_space).base(), s.end());
    return s;
}

In [ ]:
%%script false
string trim(string s) {
    return trim_left(trim_right(move(s)));
}

In [81]:
# 動くプログラムになるか
# folding
# Counting newline characters with accumulate
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

bool is_not_space(char c) {
    return (c != ' ');
} 

string trim_left(string s) {
    s.erase(s.begin(), find_if(s.begin(), s.end(), is_not_space));
    return s;
}

string trim_right(string s) {
    s.erase(find_if(s.rbegin(), s.rend(), is_not_space).base(), s.end());
    return s;
}

string trim(string s) {
    return trim_left(trim_right(move(s)));
}

int main() {
    string str = "  this is a pen  ";
    cout << "(begin)" << str << "(end)" << endl;
    cout << "(begin)" << trim(str) << "(end)" << endl;
}

Overwriting temp.cpp


In [82]:
!g++  temp.cpp; ./a.out

(begin)  this is a pen  (end)
(begin)this is a pen(end)


In [ ]:
# 2.2.4 Partitioning collections based on a predicate
# Listing 2.5 Females first

# partition(people.begin(), people.end(),is_female);

# Listing 2.6 Moving selected items to a specific point

# stable_partition(first, destination, is_not_selected);
# stable_partition(destination, last,  is_selected);

In [ ]:
# 2.2.5 Filtering and transforming
# bool is_female(const person_t& person);
# bool is_not_female(const person_t& person);
# string name(const person_t& person);

In [ ]:
# Listing 2.7 Filtering items by removing undesired ones
# people.erase(remove_if(people.begin(), people.end(),is_not_female),people.end());

In [ ]:
!g++  temp.cpp; ./a.out

# いまここ